# TD - Graph Traversal: BFS, DFS and Shortest Paths

## Objectives
- Understand and implement breadth-first search (BFS)
- Understand and implement depth-first search (DFS)
- Use shortest path algorithms
- Visualize results with Plotly on a real graph (Paris road network)

For algorithm animations: https://visualgo.net/en/sssp

## Data
We will use the Paris road network obtained via OSMnx, which retrieves OpenStreetMap data.


In [ ]:
# Installation of necessary libraries
# !pip install osmnx networkx plotly matplotlib numpy pandas

In [ ]:
import osmnx as ox
import networkx as nx
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from collections import deque
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from IPython.display import display, HTML

print("Libraries imported successfully!")
print(f"NetworkX version: {nx.__version__}")
print(f"OSMnx version: {ox.__version__}")

## 0. What is a Road Network Graph?

In [ ]:
G = ox.graph_from_place("Etretat, France", network_type="drive")
print(G)

In [ ]:
nodes_df.describe()

In [ ]:
nodes_df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
nodes_df.index.name = 'node'
nodes_df

In [ ]:
edges_df = pd.DataFrame([
    {'source': u, 'target': v, **data}
    for u, v, data in G.edges(data=True)
])
edges_df

In [ ]:
fig = plot_graph_plotly(G, title="The Road Network")
fig.show()

## 1. Loading the Paris Road Network Graph


In [ ]:
# Loading the Paris road network graph
print("Loading Paris road network...")
G = ox.graph_from_place("Paris, France", network_type="drive")

# Convert to undirected NetworkX graph to simplify algorithms
# (we can keep the directed graph if necessary)
G = G.to_undirected()

print(f"Graph loaded: {len(G.nodes())} nodes, {len(G.edges())} edges")
print(f"Graph type: {type(G)}")
print(f"Is connected: {nx.is_connected(G)}")

In [ ]:
# Select a main connected component if necessary
if not nx.is_connected(G):
    print("Graph is not connected. Selecting the largest connected component...")
    largest_cc = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_cc).copy()
    print(f"New graph: {len(G.nodes())} nodes, {len(G.edges())} edges")

# Select two nodes for our experiments
nodes = list(G.nodes())
start_node = nodes[0]
end_node = nodes[len(nodes)//4]  # A node at approximately 1/4 of the graph

print(f"\nStart node: {start_node}")
print(f"End node: {end_node}")

## 2. Utility Function for Visualization with Plotly


In [ ]:
def plot_graph_plotly(G, title="Graph", highlight_nodes=None, highlight_path=None, node_colors=None):
    """
    Visualize a graph with Plotly on a map
    
    Parameters:
    - G: NetworkX graph
    - title: chart title
    - highlight_nodes: list of nodes to highlight
    - highlight_path: list of nodes forming a path to highlight
    - node_colors: dictionary {node: color} to color nodes
    """
    # Get coordinates (longitude, latitude)
    pos = {}
    for node in G.nodes():
        if 'x' in G.nodes[node] and 'y' in G.nodes[node]:
            # OSMnx stores x=longitude, y=latitude
            pos[node] = (G.nodes[node]['x'], G.nodes[node]['y'])
        elif 'lon' in G.nodes[node] and 'lat' in G.nodes[node]:
            pos[node] = (G.nodes[node]['lon'], G.nodes[node]['lat'])
        else:
            # Fallback: use node ID as coordinates
            pos[node] = (node, 0)
    
    # Calculate map center
    if pos:
        lons = [p[0] for p in pos.values() if isinstance(p[0], (int, float))]
        lats = [p[1] for p in pos.values() if isinstance(p[1], (int, float))]
        if lons and lats:
            center_lon = sum(lons) / len(lons)
            center_lat = sum(lats) / len(lats)
        else:
            center_lon, center_lat = 2.3522, 48.8566  # Paris by default
    else:
        center_lon, center_lat = 2.3522, 48.8566
    
    fig = go.Figure()
    
    # Prepare edges with coordinates (lon, lat)
    for edge in G.edges():
        if edge[0] in pos and edge[1] in pos:
            lon0, lat0 = pos[edge[0]]
            lon1, lat1 = pos[edge[1]]
            # Check that coordinates are valid
            if (isinstance(lon0, (int, float)) and isinstance(lat0, (int, float)) and
                isinstance(lon1, (int, float)) and isinstance(lat1, (int, float))):
                fig.add_trace(go.Scattermap(
                    mode='lines',
                    lon=[lon0, lon1],
                    lat=[lat0, lat1],
                    line=dict(width=0.5, color='#888'),
                    hoverinfo='none',
                    showlegend=False
                ))
    
    # Prepare nodes
    node_lons = []
    node_lats = []
    node_text = []
    node_colors_list = []
    node_sizes = []
    
    for node in G.nodes():
        if node in pos:
            lon, lat = pos[node]
            if isinstance(lon, (int, float)) and isinstance(lat, (int, float)):
                node_lons.append(lon)
                node_lats.append(lat)
                node_text.append(f'Node {node}')
                
                # Determine color and size
                if highlight_path and node in highlight_path:
                    color = 'red'
                    size = 10
                elif highlight_nodes and node in highlight_nodes:
                    color = 'orange'
                    size = 12
                elif node_colors and node in node_colors:
                    color = node_colors[node]
                    size = 8
                else:
                    color = 'lightblue'
                    size = 4
                node_colors_list.append(color)
                node_sizes.append(size)
    
    # Add trace for nodes
    if node_lons and node_lats:
        fig.add_trace(go.Scattermap(
            mode='markers',
            lon=node_lons,
            lat=node_lats,
            text=node_text,
            hoverinfo='text',
            marker=dict(
                size=node_sizes,
                color=node_colors_list,
                opacity=0.8
            ),
            showlegend=False
        ))
    
    # Update layout with mapbox
    fig.update_layout(
        title=title,
        mapbox=dict(
            style="open-street-map",  # Use OpenStreetMap as basemap
            center=dict(lon=center_lon, lat=center_lat),
            zoom=11,  # Zoom level adapted to Paris
            bearing=0,
            pitch=0
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        height=700
    )
    
    return fig

## 3. Initial Graph Visualization


In [ ]:
# Visualization of the complete graph (sampled for performance)
sample_nodes = list(G.nodes())[::10]  # Take 1 node out of 10 for visualization
G_sample = G.subgraph(sample_nodes).copy()

fig = plot_graph_plotly(G_sample, title="Paris Road Network (sample)")
fig.show()

## 4. Breadth-First Search (BFS)

### 4.0 BFS Theory

**Breadth-First Search (BFS)** is a fundamental graph traversal algorithm that systematically explores a graph level by level.

#### Principle of Operation

BFS uses a **queue (FIFO: First In First Out)** to maintain the order of node visits:

1. **Initialization**: Start with a source node, mark it as visited and add it to the queue
2. **Exploration**: While the queue is not empty:
   - Remove the first node from the queue (dequeue)
   - Visit all its unvisited neighbors
   - Mark these neighbors as visited and add them to the end of the queue (enqueue)
3. **Result**: We obtain the order of visits level by level

#### Important Properties

- **Shortest path**: On an **unweighted** graph, BFS guarantees finding the shortest path in number of edges between two nodes
- **Data structure**: Queue (FIFO) - the first node added is the first to be processed
- **Time complexity**: O(V + E) where V is the number of nodes and E is the number of edges
- **Space complexity**: O(V) to store the queue and visited nodes

#### Applications

- Maze navigation
- Social networks: finding the shortest path between two users
- Network routing: finding the path with the least hops
- Graph connectivity verification
- Cycle detection (on undirected graphs)

#### Pseudocode

```
BFS(G, start):
    queue = [start]
    visited = {start}
    order = [start]
    parent = {start: None}
    
    while queue not empty:
        node = queue.dequeue()
        
        for neighbor in G.neighbors(node):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.enqueue(neighbor)
                order.append(neighbor)
                parent[neighbor] = node
    
    return visited, order, parent
```

#### Visual Example

For a graph with starting node A:
- Level 0: A
- Level 1: all direct neighbors of A
- Level 2: all neighbors of nodes at level 1 (not already visited)
- etc.

### 4.1 BFS Implementation


In [ ]:
def bfs(G, start, end=None):
    """
    Breadth-first search (BFS)
    
    Parameters:
    - G: NetworkX graph
    - start: starting node
    - end: destination node (optional, if None, traverses entire graph)
    
    Returns:
    - visited: set of visited nodes
    - path: path to 'end' if specified, None otherwise
    - order: order of node visits
    """
    visited = set()
    queue = deque([start])
    visited.add(start)
    order = [start]
    parent = {start: None}
    
    while queue:
        node = queue.popleft()
        
        # If we're looking for a specific node and found it
        if end is not None and node == end:
            # Reconstruct the path
            path = []
            current = end
            while current is not None:
                path.append(current)
                current = parent[current]
            return visited, list(reversed(path)), order
        
        # Traverse neighbors
        for neighbor in G.neighbors(node):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
                parent[neighbor] = node
                order.append(neighbor)
    
    # If end was specified but not found
    if end is not None:
        return visited, None, order
    
    return visited, None, order

# Test BFS
print("Running BFS...")
visited_bfs, path_bfs, order_bfs = bfs(G, start_node, end_node)

print(f"Number of visited nodes: {len(visited_bfs)}")
print(f"Path length found: {len(path_bfs) if path_bfs else 'No path'}")

if path_bfs:
    print(f"Path (first 10 nodes): {path_bfs[:10]}...")
    print(f"Path (last 10 nodes): ...{path_bfs[-10:]}")


### 4.2 BFS Visualization


In [ ]:
# Create a subgraph containing nodes visited by BFS
if path_bfs:
    nodes_in_path = set()
    for i in range(len(path_bfs) - 1):
        nodes_in_path.add(path_bfs[i])
        nodes_in_path.add(path_bfs[i+1])
        # Add immediate neighbors to see the context
        for neighbor in G.neighbors(path_bfs[i]):
            nodes_in_path.add(neighbor)

    G_bfs = G.subgraph(list(nodes_in_path)[:500]).copy()  # Limit to 500 nodes for performance

    # Create a color dictionary to show visit order
    node_colors = {}
    for i, node in enumerate(order_bfs[:100]):  # Limit for performance
        # Use a color palette
        if node in path_bfs:
            node_colors[node] = 'red'
        else:
            # Color based on visit order
            intensity = int(255 * (i / min(100, len(order_bfs))))
            node_colors[node] = f'rgb({intensity}, {255-intensity}, 200)'

    fig = plot_graph_plotly(
        G_bfs, 
        title="BFS Traversal - Paris Road Network",
        highlight_nodes=[start_node, end_node],
        highlight_path=path_bfs,
        node_colors=node_colors
    )
    fig.show()

    print(f"BFS visualization: {len(G_bfs.nodes())} nodes displayed")
else:
    print("No path found with BFS, cannot visualize")


### 4.3 BFS with NetworkX (built-in function)


In [ ]:
# Using NetworkX's built-in BFS function
try:
    bfs_edges_nx = list(nx.bfs_edges(G, source=start_node))
    bfs_tree_nx = nx.bfs_tree(G, source=start_node)
    
    print(f"NetworkX BFS - Number of edges explored: {len(bfs_edges_nx)}")
    print(f"NetworkX BFS - Tree size: {len(bfs_tree_nx.nodes())} nodes")
    
    # Shortest path with BFS (if weights are equal)
    try:
        path_bfs_nx = nx.shortest_path(G, source=start_node, target=end_node, method='bfs')
        print(f"NetworkX BFS Path - Length: {len(path_bfs_nx)}")
    except nx.NetworkXNoPath:
        print("No path found with NetworkX BFS")
        
except Exception as e:
    print(f"Error with NetworkX BFS: {e}")


## 5. Depth-First Search (DFS)

### 5.0 DFS Theory

**Depth-First Search (DFS)** is a graph traversal algorithm that explores the graph by going as far as possible along a branch before backtracking.

#### Principle of Operation

DFS uses a **stack (LIFO: Last In First Out)** or **recursion** to maintain the order of visits:

1. **Initialization**: Start with a source node, mark it as visited
2. **Recursive exploration**: For each visited node:
   - Recursively explore the first unvisited neighbor
   - Continue until reaching a node with no unvisited neighbors
   - Backtrack to explore other branches
3. **Result**: We obtain a depth-first traversal of the graph

#### Two Main Implementations

1. **Recursive DFS**: Uses the function call stack
2. **Iterative DFS**: Explicitly uses a data stack

#### Important Properties

- **Shortest path**: DFS **does NOT guarantee** finding the shortest path
- **Data structure**: Stack (LIFO) - the last node added is the first to be processed
- **Time complexity**: O(V + E) where V is the number of nodes and E is the number of edges
- **Space complexity**: O(V) for the stack (recursion or explicit stack) and visited nodes

#### Applications

- Cycle detection in a graph
- Topological sort (on directed acyclic graphs)
- Backtracking problem solving (mazes, sudoku)
- Finding strongly connected components
- Tree traversal (preorder, postorder)
- Graph algorithms (Hamiltonian path, etc.)

#### Pseudocode (recursive)

```
DFS_recursive(G, node, visited):
    visited.add(node)
    
    for neighbor in G.neighbors(node):
        if neighbor not in visited:
            DFS_recursive(G, neighbor, visited)
```

#### Pseudocode (iterative with stack)

```
DFS_iterative(G, start):
    stack = [start]
    visited = set()
    
    while stack not empty:
        node = stack.pop()
        
        if node not in visited:
            visited.add(node)
            
            for neighbor in G.neighbors(node):
                if neighbor not in visited:
                    stack.push(neighbor)
    
    return visited
```

#### Visual Example

For a graph with starting node A having neighbors B and C:
- Visit A
- Choose B and recursively explore its entire branch before returning
- Return to A and explore C and its branch

### 5.1 DFS Implementation (recursive)


In [ ]:
%%time

def dfs_recursive(G, node, visited, order, end=None, parent=None, path=None):
    """
    Recursive DFS
    
    Parameters:
    - G: NetworkX graph
    - node: current node
    - visited: set of visited nodes
    - order: list of visit order
    - end: destination node (optional)
    - parent: parent dictionary to reconstruct path
    - path: current path (to return found path)
    """
    visited.add(node)
    order.append(node)
    
    if path is None:
        path = []
    path.append(node)
    
    # If we found the target node
    if end is not None and node == end:
        return True, path
    
    # Traverse neighbors
    for neighbor in G.neighbors(node):
        if neighbor not in visited:
            if parent is not None:
                parent[neighbor] = node
            found, found_path = dfs_recursive(G, neighbor, visited, order, end, parent, path.copy())
            if found:
                return True, found_path
    
    return False, None

def dfs(G, start, end=None):
    """
    Depth-first search (DFS)
    
    Parameters:
    - G: NetworkX graph
    - start: starting node
    - end: destination node (optional)
    
    Returns:
    - visited: set of visited nodes
    - path: path to 'end' if specified, None otherwise
    - order: order of node visits
    """
    visited = set()
    order = []
    parent = {start: None}
    
    found, path = dfs_recursive(G, start, visited, order, end, parent, None)
    
    if end is not None:
        return visited, path if found else None, order
    
    return visited, None, order

# Test DFS
print("Running DFS...")
visited_dfs, path_dfs, order_dfs = dfs(G, start_node, end_node)

print(f"Number of visited nodes: {len(visited_dfs)}")
print(f"Path length found: {len(path_dfs) if path_dfs else 'No path'}")

if path_dfs:
    print(f"Path (first 10 nodes): {path_dfs[:10]}...")
    print(f"Path (last 10 nodes): ...{path_dfs[-10:]}")


### 5.2 DFS Implementation (iterative with stack)


In [ ]:
%%time

def dfs_iterative(G, start, end=None):
    """
    Iterative DFS using a stack
    
    Parameters:
    - G: NetworkX graph
    - start: starting node
    - end: destination node (optional)
    
    Returns:
    - visited: set of visited nodes
    - path: path to 'end' if specified, None otherwise
    - order: order of node visits
    """
    visited = set()
    stack = [start]
    order = []
    parent = {start: None}
    
    while stack:
        node = stack.pop()
        
        if node not in visited:
            visited.add(node)
            order.append(node)
            
            # If we found the target node
            if end is not None and node == end:
                # Reconstruct the path
                path = []
                current = end
                while current is not None:
                    path.append(current)
                    current = parent[current]
                return visited, list(reversed(path)), order
            
            # Add neighbors to stack (in reverse order for consistency)
            neighbors = list(G.neighbors(node))
            neighbors.reverse()
            for neighbor in neighbors:
                if neighbor not in visited:
                    stack.append(neighbor)
                    if neighbor not in parent:
                        parent[neighbor] = node
    
    # If end was specified but not found
    if end is not None:
        return visited, None, order
    
    return visited, None, order

# Test iterative DFS
print("Running iterative DFS...")
visited_dfs_iter, path_dfs_iter, order_dfs_iter = dfs_iterative(G, start_node, end_node)

print(f"Number of visited nodes: {len(visited_dfs_iter)}")
print(f"Path length found: {len(path_dfs_iter) if path_dfs_iter else 'No path'}")

if path_dfs_iter:
    print(f"Path (first 10 nodes): {path_dfs_iter[:10]}...")
    print(f"Path (last 10 nodes): ...{path_dfs_iter[-10:]}")


### 5.3 DFS Visualization


In [ ]:
# Create a subgraph containing nodes visited by DFS
if path_dfs_iter:
    nodes_in_path_dfs = set()
    for i in range(len(path_dfs_iter) - 1):
        nodes_in_path_dfs.add(path_dfs_iter[i])
        nodes_in_path_dfs.add(path_dfs_iter[i+1])
        for neighbor in G.neighbors(path_dfs_iter[i]):
            nodes_in_path_dfs.add(neighbor)
    
    G_dfs = G.subgraph(list(nodes_in_path_dfs)[:500]).copy()
    
    # Create a color dictionary to show visit order
    node_colors_dfs = {}
    for i, node in enumerate(order_dfs_iter[:100]):
        if node in path_dfs_iter:
            node_colors_dfs[node] = 'red'
        else:
            intensity = int(255 * (i / min(100, len(order_dfs_iter))))
            node_colors_dfs[node] = f'rgb({intensity}, 200, {255-intensity})'
    
    fig = plot_graph_plotly(
        G_dfs, 
        title="DFS Traversal - Paris Road Network",
        highlight_nodes=[start_node, end_node],
        highlight_path=path_dfs_iter,
        node_colors=node_colors_dfs
    )
    fig.show()
    
    print(f"DFS visualization: {len(G_dfs.nodes())} nodes displayed")
else:
    print("No path found with DFS")


### 5.4 DFS with NetworkX (built-in function)


In [ ]:
%%time
# Using NetworkX's built-in DFS function
try:
    dfs_edges_nx = list(nx.dfs_edges(G, source=start_node))
    dfs_tree_nx = nx.dfs_tree(G, source=start_node)
    
    print(f"NetworkX DFS - Number of edges explored: {len(dfs_edges_nx)}")
    print(f"NetworkX DFS - Tree size: {len(dfs_tree_nx.nodes())} nodes")
    
except Exception as e:
    print(f"Error with NetworkX DFS: {e}")


## 6. BFS vs DFS Comparison

| Characteristic | BFS | DFS |
|----------------|-----|-----|
| Structure used | Queue (FIFO) | Stack (LIFO) |
| Traversal type | Level by level | Depth-first |
| Shortest path (unweighted graph) | Yes | Not guaranteed |
| Memory usage | Higher (keeps all levels) | Lower |
| Time complexity | O(V + E) | O(V + E) |


In [ ]:
# Compare results
print("=== BFS vs DFS Comparison ===\n")

print(f"BFS:")
print(f"  - Visited nodes: {len(visited_bfs)}")
print(f"  - Path length: {len(path_bfs) if path_bfs else 'N/A'}")

print(f"\nDFS:")
print(f"  - Visited nodes: {len(visited_dfs_iter)}")
print(f"  - Path length: {len(path_dfs_iter) if path_dfs_iter else 'N/A'}")

if path_bfs and path_dfs_iter:
    print(f"\nBFS finds a path of {len(path_bfs)} nodes")
    print(f"DFS finds a path of {len(path_dfs_iter)} nodes")
    print(f"Difference: {abs(len(path_bfs) - len(path_dfs_iter))} nodes")
    print(f"\nNote: On an unweighted graph, BFS guarantees the shortest path in number of nodes.")


## 7. Shortest Path Algorithms

### 7.1 Dijkstra (shortest path with weights)

#### 7.1.0 Dijkstra's Algorithm Theory

**Dijkstra's algorithm** is a fundamental shortest path algorithm that finds the minimum-weight path from a source node to all other nodes in a weighted graph.

#### Principle of Operation

Dijkstra's algorithm uses a **greedy approach** with a priority queue:

1. **Initialization**: 
   - Set distance to source node = 0
   - Set distance to all other nodes = ∞ (infinity)
   - Create a priority queue (min-heap) containing all nodes

2. **Iterative Process**:
   - Extract the node with the minimum distance from the priority queue
   - For each neighbor of this node:
     - Calculate tentative distance = current distance + edge weight
     - If tentative distance < current distance to neighbor:
       - Update neighbor's distance
       - Update priority queue
   - Mark the current node as processed

3. **Termination**: When the priority queue is empty or when the target node is processed

#### Important Properties

- **Optimality**: Guarantees finding the shortest path when all edge weights are **non-negative**
- **Data structure**: Priority queue (min-heap) to efficiently select the next node
- **Time complexity**: 
  - O((V + E) log V) with binary heap
  - O(V log V + E) with Fibonacci heap (optimal)
- **Space complexity**: O(V) to store distances and the priority queue
- **Restriction**: **Does NOT work with negative edge weights** (use Bellman-Ford for that)

#### Why Not Negative Weights?

If negative weights are allowed, a shorter path could be found by repeatedly going through a negative cycle, reducing the distance indefinitely. Dijkstra assumes that once a node is processed, its shortest distance is final, which is only true for non-negative weights.

#### Comparison with BFS

| Characteristic | BFS | Dijkstra |
|----------------|-----|----------|
| Graph type | Unweighted | Weighted |
| Weights | All equal (1) | Non-negative |
| Data structure | Queue (FIFO) | Priority queue |
| Shortest path | In number of edges | In total weight |
| Time complexity | O(V + E) | O((V + E) log V) |

#### Applications

- **GPS navigation**: Finding shortest routes (time or distance)
- **Network routing**: Finding optimal paths in computer networks
- **Social networks**: Finding minimum cost connections
- **Game development**: Pathfinding in strategy games
- **Resource allocation**: Minimizing costs in logistics

#### Pseudocode

```
Dijkstra(G, source):
    dist = [∞ for all nodes]
    dist[source] = 0
    priority_queue = [(0, source)]  # (distance, node)
    visited = set()
    
    while priority_queue not empty:
        current_dist, current = priority_queue.extract_min()
        
        if current in visited:
            continue
        
        visited.add(current)
        
        for neighbor in G.neighbors(current):
            weight = G.edge_weight(current, neighbor)
            tentative_dist = current_dist + weight
            
            if tentative_dist < dist[neighbor]:
                dist[neighbor] = tentative_dist
                priority_queue.insert((tentative_dist, neighbor))
    
    return dist
```

Dijkstra's algorithm finds the shortest path in a weighted graph with non-negative edge weights.


In [ ]:
%%time
# Check if the graph has length/distance attributes
sample_edge = list(G.edges(data=True))[0]
print(f"Sample edge with attributes: {sample_edge}")

# Use length if available, otherwise use a calculated distance
if 'length' in sample_edge[2]:
    weight = 'length'
    print("\nUsing 'length' attribute for weights")
elif 'weight' in sample_edge[2]:
    weight = 'weight'
    print("\nUsing 'weight' attribute for weights")
else:
    # Calculate geodesic distance if lon/lat available
    weight = None
    print("\nCalculating geodesic distance for weights")
    
    def calculate_distance(u, v):
        """Calculate geodesic distance between two nodes"""
        try:
            if 'x' in G.nodes[u] and 'y' in G.nodes[u]:
                x1, y1 = G.nodes[u]['x'], G.nodes[u]['y']
                x2, y2 = G.nodes[v]['x'], G.nodes[v]['y']
            elif 'lon' in G.nodes[u] and 'lat' in G.nodes[u]:
                x1, y1 = G.nodes[u]['lon'], G.nodes[u]['lat']
                x2, y2 = G.nodes[v]['lon'], G.nodes[v]['lat']
            else:
                return 1.0  # Default distance
            
            # Simple euclidean distance (or use geopy for real distance)
            return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        except:
            return 1.0
    
    # Add calculated weights to graph
    for u, v, d in G.edges(data=True):
        d['calculated_length'] = calculate_distance(u, v)
    weight = 'calculated_length'


In [ ]:
# Calculate shortest path with Dijkstra
try:
    print("Calculating shortest path with Dijkstra...")
    path_dijkstra = nx.shortest_path(G, source=start_node, target=end_node, weight=weight, method='dijkstra')
    length_dijkstra = nx.shortest_path_length(G, source=start_node, target=end_node, weight=weight, method='dijkstra')
    
    print(f"Dijkstra path - Length: {len(path_dijkstra)} nodes")
    print(f"Total distance: {length_dijkstra:.2f} meters" if weight else f"Total distance: {length_dijkstra:.2f}")
    print(f"Path (first 10 nodes): {path_dijkstra[:10]}...")
    print(f"Path (last 10 nodes): ...{path_dijkstra[-10:]}")
    
except Exception as e:
    print(f"Error with Dijkstra: {e}")
    path_dijkstra = None
    length_dijkstra = None


### 7.2 A* (A-star) - Heuristic for optimization

#### 7.2.0 A* Algorithm Theory

The **A*** algorithm (pronounced "A-star") is an improvement of Dijkstra's algorithm that uses a **heuristic function** to guide the search towards the goal, thus reducing the number of nodes explored.

#### Principle of Operation

A* combines two pieces of information to decide which node to explore next:

1. **g(n)**: Actual cost of the path from the start node to node n
2. **h(n)**: Heuristic estimate of the remaining cost from node n to the goal

The evaluation function is: **f(n) = g(n) + h(n)**

The algorithm always explores the node with the smallest f(n) value.

#### Heuristic Function

A good heuristic must be:

- **Admissible**: Never overestimates the actual cost to the goal (h(n) ≤ actual cost)
- **Consistent/Monotone**: For any node n and its successor n', h(n) ≤ c(n,n') + h(n')

#### Important Properties

- **Optimality**: If the heuristic is admissible, A* guarantees finding the optimal path
- **Efficiency**: Generally explores fewer nodes than Dijkstra, especially with a good heuristic
- **Time complexity**: O((V + E) log V) in worst case (like Dijkstra), but often better in practice
- **Space complexity**: O(V) to store explored nodes

#### Comparison with Dijkstra

| Characteristic | Dijkstra | A* |
|----------------|----------|-----|
| Heuristic | No | Yes |
| Exploration | Uniform (in circles) | Goal-directed |
| Nodes explored | More | Fewer (with good heuristic) |
| Optimality | Yes | Yes (if heuristic admissible) |

#### Common Heuristics

1. **Euclidean distance**: h(n) = euclidean distance between n and goal (for geographic graphs)
2. **Manhattan distance**: h(n) = |x_n - x_goal| + |y_n - y_goal| (for grids)
3. **Chebyshev distance**: For diagonal movements allowed

#### Applications

- Video games: character pathfinding
  - Examples: Starcraft, Age of Empires, strategy games
- Robotics: trajectory planning
- GPS navigation (variants with constraints)
- Artificial intelligence: puzzle solving, state space search

#### Pseudocode

```
A_star(G, start, goal, heuristic):
    open_set = [(heuristic(start, goal), start)]  # (f, node)
    g_score = {node: ∞ for node in G.nodes}
    g_score[start] = 0
    f_score = {node: ∞ for node in G.nodes}
    f_score[start] = heuristic(start, goal)
    parent = {start: None}
    
    while open_set not empty:
        current = open_set.extract_min()  # node with smallest f
        
        if current == goal:
            return reconstruct_path(parent, current)
        
        for neighbor in G.neighbors(current):
            tentative_g = g_score[current] + weight(current, neighbor)
            
            if tentative_g < g_score[neighbor]:
                parent[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                open_set.insert_or_update((f_score[neighbor], neighbor))
    
    return None  # No path found
```

The A* algorithm is an improvement of Dijkstra that uses a heuristic to guide the search.


In [ ]:
%%time
# Heuristic function for A* (euclidean distance to goal)
def heuristic(u, v):
    """Heuristic: euclidean distance between two nodes"""
    try:
        if 'x' in G.nodes[u] and 'y' in G.nodes[u]:
            x1, y1 = G.nodes[u]['x'], G.nodes[u]['y']
            x2, y2 = G.nodes[v]['x'], G.nodes[v]['y']
        elif 'lon' in G.nodes[u] and 'lat' in G.nodes[u]:
            x1, y1 = G.nodes[u]['lon'], G.nodes[u]['lat']
            x2, y2 = G.nodes[v]['lon'], G.nodes[v]['lat']
        else:
            return 0
        
        return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    except:
        return 0

# Calculate shortest path with A*
try:
    print("Calculating shortest path with A*...")
    path_astar = nx.astar_path(G, source=start_node, target=end_node, weight=weight, heuristic=heuristic)
    length_astar = nx.astar_path_length(G, source=start_node, target=end_node, weight=weight, heuristic=heuristic)
    
    print(f"A* path - Length: {len(path_astar)} nodes")
    print(f"Total distance: {length_astar:.2f} meters" if weight else f"Total distance: {length_astar:.2f}")
    print(f"Path (first 10 nodes): {path_astar[:10]}...")
    print(f"Path (last 10 nodes): ...{path_astar[-10:]}")
    
except Exception as e:
    print(f"Error with A*: {e}")
    path_astar = None
    length_astar = None


### 7.3 Bellman-Ford Algorithm

#### 7.3.0 Bellman-Ford Algorithm Theory

The **Bellman-Ford algorithm** is a shortest path algorithm that can handle graphs with **negative edge weights** and can detect **negative cycles**.

#### Principle of Operation

Bellman-Ford uses a **relaxation technique** that repeatedly updates distance estimates:

1. **Initialization**: 
   - Set distance to source node = 0
   - Set distance to all other nodes = ∞ (infinity)

2. **Relaxation Phase** (repeated V-1 times):
   - For each edge (u, v) with weight w:
     - If distance[u] + w < distance[v]:
       - Update distance[v] = distance[u] + w
       - Update predecessor[v] = u

3. **Negative Cycle Detection** (one additional pass):
   - For each edge (u, v) with weight w:
     - If distance[u] + w < distance[v]:
       - **Negative cycle detected** (no shortest path exists)

#### Why V-1 Iterations?

In a graph with V nodes, the longest simple path can have at most V-1 edges. After V-1 iterations of relaxation, all shortest paths should be found. If we can still relax an edge after V-1 iterations, it means there's a negative cycle.

#### Important Properties

- **Handles negative weights**: Unlike Dijkstra, Bellman-Ford works with negative edge weights
- **Detects negative cycles**: Can identify if a negative cycle exists in the graph
- **Optimality**: Guarantees finding the shortest path if no negative cycles exist
- **Time complexity**: O(V × E) - slower than Dijkstra
- **Space complexity**: O(V) to store distances
- **Best for**: Sparse graphs with negative weights or when negative cycles need detection

#### Comparison with Dijkstra

| Characteristic | Dijkstra | Bellman-Ford |
|----------------|----------|--------------|
| Negative weights | ❌ No | ✅ Yes |
| Negative cycles | N/A | ✅ Detects |
| Time complexity | O((V + E) log V) | O(V × E) |
| Best for | Dense graphs, non-negative weights | Sparse graphs, negative weights |
| Data structure | Priority queue | Simple arrays |

#### Applications

- **Network routing protocols**: RIP (Routing Information Protocol) uses a variant
- **Financial modeling**: Detecting arbitrage opportunities (negative cycles)
- **Game development**: Currency exchange with negative conversion rates
- **Signal processing**: Finding shortest paths with negative correlations
- **Graph analysis**: Cycle detection in weighted graphs

#### Pseudocode

```
Bellman_Ford(G, source):
    dist = [∞ for all nodes]
    dist[source] = 0
    predecessor = [None for all nodes]
    
    # Relaxation phase: V-1 iterations
    for i in range(V - 1):
        for each edge (u, v) with weight w in G:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                predecessor[v] = u
    
    # Negative cycle detection: one more iteration
    for each edge (u, v) with weight w in G:
        if dist[u] + w < dist[v]:
            return "Negative cycle detected!"
    
    return dist, predecessor
```

The Bellman-Ford algorithm is more versatile than Dijkstra but slower, making it ideal for graphs with negative weights or when cycle detection is needed.


In [ ]:
# Calculate shortest path with Bellman-Ford
try:
    print("Calculating shortest path with Bellman-Ford...")
    
    # Convert graph to directed if needed for Bellman-Ford
    # NetworkX bellman_ford requires a directed graph
    if not G.is_directed():
        G_directed = G.to_directed()
    else:
        G_directed = G
    
    # Use NetworkX's Bellman-Ford implementation
    path_bellman = nx.shortest_path(G_directed, source=start_node, target=end_node, weight=weight, method='bellman-ford')
    length_bellman = nx.shortest_path_length(G_directed, source=start_node, target=end_node, weight=weight, method='bellman-ford')
    
    print(f"Bellman-Ford path - Length: {len(path_bellman)} nodes")
    print(f"Total distance: {length_bellman:.2f} meters" if weight else f"Total distance: {length_bellman:.2f}")
    print(f"Path (first 10 nodes): {path_bellman[:10]}...")
    print(f"Path (last 10 nodes): ...{path_bellman[-10:]}")
    
except nx.NetworkXUnbounded:
    print("Error: Negative cycle detected in the graph!")
    path_bellman = None
    length_bellman = None
except Exception as e:
    print(f"Error with Bellman-Ford: {e}")
    path_bellman = None
    length_bellman = None


### 7.4 Comparison of shortest path algorithms


In [ ]:
import time

print("=== Algorithm Comparison Benchmark ===\n")

# Prepare directed graph for Bellman-Ford if needed
if not G.is_directed():
    G_directed = G.to_directed()
else:
    G_directed = G

# Build algorithms based on weight availability
algorithms = {}

if weight:
    algorithms['Dijkstra'] = lambda: nx.shortest_path(G, start_node, end_node, weight=weight, method='dijkstra')
    algorithms['A*'] = lambda: nx.astar_path(G, start_node, end_node, weight=weight, heuristic=heuristic)
    algorithms['Bellman-Ford'] = lambda: nx.shortest_path(G_directed, start_node, end_node, weight=weight, method='bellman-ford')
else:
    # If no weights, Dijkstra is equivalent to BFS
    algorithms['Dijkstra (no weights)'] = lambda: nx.shortest_path(G, start_node, end_node, method='dijkstra')

results = {}
for name, func in algorithms.items():
    try:
        start_time = time.time()
        path = func()
        elapsed = time.time() - start_time
        length = len(path)
        
        if weight:
            # Calculate path length using the same method
            if name == 'Bellman-Ford':
                path_length = nx.shortest_path_length(G_directed, start_node, end_node, weight=weight, method='bellman-ford')
            elif name == 'A*':
                path_length = nx.astar_path_length(G, start_node, end_node, weight=weight, heuristic=heuristic)
            else:
                path_length = nx.shortest_path_length(G, start_node, end_node, weight=weight, method='dijkstra')
            
            results[name] = {
                'path': path,
                'node_count': length,
                'distance': path_length,
                'time_ms': elapsed * 1000,
                'time_s': elapsed
            }
        else:
            results[name] = {
                'path': path,
                'node_count': length,
                'distance': None,
                'time_ms': elapsed * 1000,
                'time_s': elapsed
            }
    except nx.NetworkXUnbounded:
        results[name] = {
            'path': None,
            'node_count': None,
            'distance': None,
            'time_ms': None,
            'time_s': None,
            'error': 'Negative cycle detected'
        }
    except Exception as e:
        results[name] = {
            'path': None,
            'node_count': None,
            'distance': None,
            'time_ms': None,
            'time_s': None,
            'error': str(e)
        }

# Display results in a formatted table
print("Benchmark Results:\n")
print("=" * 80)
print(f"{'Algorithm':<20} {'Time (ms)':<15} {'Time (s)':<15} {'Nodes':<10} {'Distance (m)':<15}")
print("=" * 80)

for name, result in results.items():
    if 'error' in result:
        print(f"{name:<20} {'ERROR':<15} {'ERROR':<15} {'ERROR':<10} {result['error']:<15}")
    else:
        time_ms = f"{result['time_ms']:.2f}" if result['time_ms'] is not None else "N/A"
        time_s = f"{result['time_s']:.6f}" if result['time_s'] is not None else "N/A"
        nodes = f"{result['node_count']}" if result['node_count'] is not None else "N/A"
        distance = f"{result['distance']:.2f}" if result['distance'] is not None else "N/A"
        print(f"{name:<20} {time_ms:<15} {time_s:<15} {nodes:<10} {distance:<15}")

print("=" * 80)

# Summary statistics
valid_results = {k: v for k, v in results.items() if 'error' not in v and v['time_ms'] is not None}
if valid_results:
    times = [v['time_ms'] for v in valid_results.values()]
    fastest = min(valid_results.items(), key=lambda x: x[1]['time_ms'])
    slowest = max(valid_results.items(), key=lambda x: x[1]['time_ms'])
    
    print(f"\nSummary:")
    print(f"  - Fastest algorithm: {fastest[0]} ({fastest[1]['time_ms']:.2f} ms)")
    print(f"  - Slowest algorithm: {slowest[0]} ({slowest[1]['time_ms']:.2f} ms)")
    print(f"  - Speedup factor: {slowest[1]['time_ms'] / fastest[1]['time_ms']:.2f}x")
    
    # Check if all algorithms found the same path length (for weighted graphs)
    if weight and all(v['distance'] is not None for v in valid_results.values()):
        distances = [v['distance'] for v in valid_results.values()]
        if len(set(distances)) == 1:
            print(f"  - All algorithms found the same optimal distance: {distances[0]:.2f} m")
        else:
            print(f"  - Distance variation detected (check implementations)")
            for name, result in valid_results.items():
                print(f"    {name}: {result['distance']:.2f} m")


### 7.5 Visualization of shortest paths


In [ ]:
import plotly.graph_objects as go

if path_dijkstra:
    all_path_nodes = set()
    for path in [path_bfs, path_dfs_iter, path_dijkstra, path_astar, path_bellman if 'path_bellman' in globals() else None]:
        if path:
            all_path_nodes.update(path)
            for node in path[:50]:
                for neighbor in list(G.neighbors(node))[:3]:
                    all_path_nodes.add(neighbor)
    
    G_paths = G.subgraph(list(all_path_nodes)[:1000]).copy()

    fig = go.Figure()

    pos = {}
    for node in G_paths.nodes():
        data = G_paths.nodes[node]
        if 'x' in data and 'y' in data:
            pos[node] = (data['x'], data['y'])
        elif 'lon' in data and 'lat' in data:
            pos[node] = (data['lon'], data['lat'])
        else:
            pos[node] = (node, 0)

    # --- Calcul du centre ---
    path_coords = []
    for path in [path_dijkstra, path_astar, path_bfs, path_bellman if 'path_bellman' in globals() else None]:
        if path:
            for node in path:
                if node in pos:
                    path_coords.append(pos[node])
    if path_coords:
        center_lon = sum(c[0] for c in path_coords) / len(path_coords)
        center_lat = sum(c[1] for c in path_coords) / len(path_coords)
    else:
        center_lon, center_lat = 2.3522, 48.8566

    # --- Arêtes du graphe ---
    edge_lons, edge_lats = [], []
    for u, v in G_paths.edges():
        if u in pos and v in pos:
            lon0, lat0 = pos[u]
            lon1, lat1 = pos[v]
            edge_lons.extend([lon0, lon1, None])
            edge_lats.extend([lat0, lat1, None])
    fig.add_trace(go.Scattermap(
        mode='lines',
        lon=edge_lons, lat=edge_lats,
        line=dict(width=0.5, color='lightgray'),
        hoverinfo='none',
        name='Réseau routier'
    ))

    # --- Dijkstra ---
    if path_dijkstra:
        lons = [pos[n][0] for n in path_dijkstra if n in pos]
        lats = [pos[n][1] for n in path_dijkstra if n in pos]
        fig.add_trace(go.Scattermap(
            mode='lines+markers',
            lon=lons, lat=lats,
            line=dict(width=4, color='blue'),
            marker=dict(size=6, color='blue', opacity=0.7),
            name=f'Dijkstra ({len(path_dijkstra)} nœuds)'
        ))

    # --- A* ---
    if path_astar:
        lons = [pos[n][0] for n in path_astar if n in pos]
        lats = [pos[n][1] for n in path_astar if n in pos]
        fig.add_trace(go.Scattermap(
            mode='lines+markers',
            lon=lons, lat=lats,
            line=dict(width=4, color='green'),
            marker=dict(size=6, color='green', opacity=0.7),
            name=f'A* ({len(path_astar)} nœuds)'
        ))

    # --- BFS ---
    if path_bfs:
        lons = [pos[n][0] for n in path_bfs if n in pos]
        lats = [pos[n][1] for n in path_bfs if n in pos]
        fig.add_trace(go.Scattermap(
            mode='lines+markers',
            lon=lons, lat=lats,
            line=dict(width=3, color='red'),
            marker=dict(size=5, color='red', opacity=0.7),
            name=f'BFS ({len(path_bfs)} nœuds)'
        ))

    # --- Bellman-Ford ---
    if 'path_bellman' in globals() and path_bellman:
        lons = [pos[n][0] for n in path_bellman if n in pos]
        lats = [pos[n][1] for n in path_bellman if n in pos]
        fig.add_trace(go.Scattermap(
            mode='lines+markers',
            lon=lons, lat=lats,
            line=dict(width=3, color='purple'),
            marker=dict(size=5, color='purple', opacity=0.7),
            name=f'Bellman-Ford ({len(path_bellman)} nœuds)'
        ))

    # --- Points de départ et d’arrivée ---
    if start_node in pos and end_node in pos:
        slon, slat = pos[start_node]
        elon, elat = pos[end_node]
        fig.add_trace(go.Scattermap(
            mode='markers',
            lon=[slon], lat=[slat],
            marker=dict(size=18, color='orange'),
            name='Départ'
        ))
        fig.add_trace(go.Scattermap(
            mode='markers',
            lon=[elon], lat=[elat],
            marker=dict(size=18, color='purple'),
            name='Arrivée'
        ))

    # --- Layout ---
    fig.update_layout(
        title="Comparaison des plus courts chemins - Paris",
        showlegend=True,
        map=dict(
            style="open-street-map",
            center=dict(lon=center_lon, lat=center_lat),
            zoom=12
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        height=800
    )

    fig.show()


## 8. Summary

### Key Points to Remember:

1. **BFS (Breadth-First Search)** :
   - Explores level by level
   - Guarantees shortest path in number of nodes (unweighted graph)
   - Uses a queue (FIFO)

2. **DFS (Depth-First Search)** :
   - Explores in depth before backtracking
   - Does not guarantee shortest path
   - Uses a stack (LIFO) or recursion

3. **Dijkstra** :
   - Finds shortest path in a weighted graph
   - Requires positive weights
   - Complexity: O((V + E) log V) with good implementation

4. **A*** :
   - Improvement of Dijkstra with heuristic
   - Faster than Dijkstra in most cases
   - Optimal if heuristic is admissible (never overestimates)

5. **Bellman-Ford** :
   - Finds shortest path in weighted graphs with negative weights
   - Can detect negative cycles
   - Complexity: O(V × E) - slower than Dijkstra but more versatile
   - Best for graphs with negative weights or when cycle detection is needed

### Practical Applications:
- GPS navigation (routing)
- Social networks (finding connections)
- Artificial intelligence (games, planning)
- Network analysis (Internet, transportation)


## 9. Algorithmic Complexity Overview

Below is a practical summary of the asymptotic time and space complexities of the algorithms studied in this notebook. Remember that V is the number of vertices (nodes) and E is the number of edges.

- **BFS**
  - Time: O(V + E) — visits each node and edge at most once
  - Space: O(V) — queue + visited/parent structures
  - Notes: Optimal on unweighted graphs (fewest edges)

- **DFS**
  - Time: O(V + E) — explores each node and edge at most once
  - Space: O(V) — recursion or explicit stack + visited
  - Notes: Not guaranteed to find shortest paths

- **Dijkstra** (non-negative weights)
  - Time: O((V + E) log V) with a binary heap priority queue; O(V log V + E) commonly written
  - Space: O(V + E) — distances, parents, priority queue bookkeeping
  - Notes: Optimal shortest paths with non-negative weights

- **A*** (with admissible heuristic)
  - Time: Worst-case like Dijkstra O((V + E) log V), but often explores fewer nodes in practice
  - Space: O(V + E) — open/closed sets, scores, parents
  - Notes: Goal-directed; optimal if heuristic is admissible (and consistent for efficiency)

- **Bellman–Ford** (handles negative weights, detects negative cycles)
  - Time: O(V × E)
  - Space: O(V) — distances + predecessors
  - Notes: Slower but more general; can detect negative cycles

The plot below uses an illustrative graph size to compare how these complexities scale. It is not a benchmark of your current graph, but a visualization of growth rates under typical assumptions.


In [ ]:
# 9.1 Complexity Comparison Plot (illustrative)
import math
import numpy as np
import matplotlib.pyplot as plt

# Illustrative graph size to visualize growth (not a benchmark of your current graph)
V = 5_000
E = 15_000

# Helper functions for growth estimates
logV = math.log(max(V, 2), 2)  # log2(V)

# Time complexity proxies (scaled values just for visualization)
time_complexity = {
    'BFS': V + E,                      # O(V + E)
    'DFS': V + E,                      # O(V + E)
    'Dijkstra': (V + E) * logV,        # O((V + E) log V)
    'A*': (V + E) * logV,              # worst-case similar to Dijkstra
    'Bellman-Ford': V * E,             # O(V × E)
}

# Space complexity proxies
space_complexity = {
    'BFS': V,                # O(V)
    'DFS': V,                # O(V)
    'Dijkstra': V + E,       # O(V + E)
    'A*': V + E,             # O(V + E)
    'Bellman-Ford': V,       # O(V)
}

algs = list(time_complexity.keys())
T = np.array([time_complexity[a] for a in algs], dtype=float)
S = np.array([space_complexity[a] for a in algs], dtype=float)

# Normalize for readable plotting (0..1)
T_norm = T / T.max()
S_norm = S / S.max()

x = np.arange(len(algs))
width = 0.38

plt.figure(figsize=(10, 5))
plt.bar(x - width/2, T_norm, width, label='Time (normalized)')
plt.bar(x + width/2, S_norm, width, label='Space (normalized)')
plt.xticks(x, algs)
plt.ylim(0, 1.1)
plt.ylabel('Normalized cost (0..1)')
plt.title('Algorithmic Complexity (Illustrative, V=5,000, E=15,000)')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.show()

print('Note: Bars are normalized to compare growth orders visually.\n' 
      'Bellman–Ford appears much larger in time due to O(V×E).\n' 
      'A* worst-case matches Dijkstra, but is often faster in practice with a good heuristic.')


In [ ]:
# 9.2 Theoretical Time Complexity Curves (sparse graph E = c·V)
import numpy as np
import matplotlib.pyplot as plt
import math

# Vertex range (log-spaced) for smooth curves
V_range = np.logspace(2, 6, 300)  # from 1e2 to 1e6
c = 3.0  # sparsity constant so E ≈ c·V
E_range = c * V_range

# Define theoretical growth (ignore constant factors)
BFS = V_range + E_range                  # O(V + E) ~ O(V) when E ~ c·V
DFS = V_range + E_range                  # O(V + E)
Dijkstra = (V_range + E_range) * np.log2(V_range)  # O((V + E) log V) ~ O(V log V)
A_star = Dijkstra                        # worst-case like Dijkstra
BellmanFord = V_range * E_range          # O(V × E) ~ O(V^2) when E ~ c·V

plt.figure(figsize=(10, 6))
plt.loglog(V_range, BFS, label='BFS (O(V+E))', linewidth=2)
plt.loglog(V_range, DFS, label='DFS (O(V+E))', linewidth=2)
plt.loglog(V_range, Dijkstra, label='Dijkstra (O((V+E) log V))', linewidth=2)
plt.loglog(V_range, A_star, label='A* (worst-case ~ Dijkstra)', linewidth=2)
plt.loglog(V_range, BellmanFord, label='Bellman–Ford (O(V×E))', linewidth=2)

plt.title('Theoretical Time Complexity (Assuming Sparse Graph E = c·V)')
plt.xlabel('Number of vertices V (log scale)')
plt.ylabel('Operation count (log scale)')
plt.grid(True, which='both', linestyle='--', alpha=0.4)
plt.legend()
plt.show()

print('Curves assume a sparse graph with E = c·V (c =', c, ').')
print('On dense graphs, E can be O(V^2), making O(V+E) ~ O(V^2) and \n' 
      'Dijkstra/A* ~ O(V^2 log V), while Bellman–Ford becomes O(V^3).')
